In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=False,
    c1=1e-4,
    tau=0.5,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.12559948861598969
epoch:  1, loss: 0.035581815987825394
epoch:  2, loss: 0.033347368240356445
epoch:  3, loss: 0.025747235864400864
epoch:  4, loss: 0.015934277325868607
epoch:  5, loss: 0.011155315674841404
epoch:  6, loss: 0.010485287755727768
epoch:  7, loss: 0.00880791898816824
epoch:  8, loss: 0.00858562346547842
epoch:  9, loss: 0.008376938290894032
epoch:  10, loss: 0.008293519727885723
epoch:  11, loss: 0.008150013163685799
epoch:  12, loss: 0.008119949139654636
epoch:  13, loss: 0.008109564892947674
epoch:  14, loss: 0.008050483651459217
epoch:  15, loss: 0.00801115296781063
epoch:  16, loss: 0.007957524619996548
epoch:  17, loss: 0.007782727479934692
epoch:  18, loss: 0.00763406977057457
epoch:  19, loss: 0.007101581897586584
epoch:  20, loss: 0.006993509363383055
epoch:  21, loss: 0.006554123014211655
epoch:  22, loss: 0.006508281454443932
epoch:  23, loss: 0.006059879902750254
epoch:  24, loss: 0.005971224047243595
epoch:  25, loss: 0.005427566822618246
e

In [6]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9844142734213406
Test metrics:  R2 = 0.9803881135091433


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].cpu().detach().numpy().item()))

    if patience <= 0:
        break

epoch:  0

/home/eugeniolr/Documents/invAI/torch_numopt/src/torch_numopt/utils.py:33: UserWarning: torch.linalg.svd: During SVD computation with the selected cusolver driver, batches 0 failed to converge. A more accurate method will be used to compute the SVD as a fallback. Check doc at https://pytorch.org/docs/stable/generated/torch.linalg.svd.html (Triggered internally at /pytorch/aten/src/ATen/native/cuda/linalg/BatchLinearAlgebraLib.cpp:690.)
  U, S, Vt = torch.linalg.svd(mat)


, loss: 0.4110935926437378
epoch:  1, loss: 0.31154173612594604
epoch:  2, loss: 0.02129548043012619
epoch:  3, loss: 0.012182684615254402
epoch:  4, loss: 0.01039897184818983
epoch:  5, loss: 0.00929376482963562
epoch:  6, loss: 0.00858816597610712
epoch:  7, loss: 0.007958177477121353
epoch:  8, loss: 0.007371501065790653
epoch:  9, loss: 0.00693253381177783
epoch:  10, loss: 0.006589781027287245
epoch:  11, loss: 0.006199134513735771
epoch:  12, loss: 0.005843295715749264
epoch:  13, loss: 0.005442881025373936
epoch:  14, loss: 0.005006241612136364
epoch:  15, loss: 0.004628636874258518
epoch:  16, loss: 0.0043015433475375175
epoch:  17, loss: 0.004032894968986511
epoch:  18, loss: 0.0038242631126195192
epoch:  19, loss: 0.003657487453892827
epoch:  20, loss: 0.00351967616006732
epoch:  21, loss: 0.0034084634389728308
epoch:  22, loss: 0.003318477887660265
epoch:  23, loss: 0.0032388572581112385
epoch:  24, loss: 0.0031822349410504103
epoch:  25, loss: 0.0029739998281002045
epoch:  

In [8]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9931194738836279
Test metrics:  R2 = 0.9917655275459792
